In [1]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

warnings.filterwarnings('ignore', category=RuntimeWarning)

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13019884996816672404, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3148847513
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7735819955652830624
 physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [1]:
import pandas as pd 
import numpy as np

data1=pd.read_csv("data.tsv", sep='\t',names=["sno","question","answer","correct","sno"])
len(list(data1["answer"]))

data1.head()

C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\pandas\io\parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


,sno,question,answer,correct,sno.1
0,0,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,0,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,0,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,0,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,0,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


In [1]:
data_try=pd.DataFrame()
data_try=data1[0:1000]
data_try.to_csv("train.csv")

NameError: name 'pd' is not defined

In [96]:
import pandas as pd 
import numpy as np
data_try=pd.read_csv("train.csv")


In [73]:
data_try=data_try[0:100]

In [16]:
def make_sequences(texts,
                   training_length=50,
                   lower=True,
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    """Turn a set of texts into sequences of integers"""

    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    # Convert text to sequences of integers
    sequences = tokenizer.texts_to_sequences(texts)

    # Limit to sequences with more than training length tokens
    seq_lengths = [len(x) for x in sequences]
    over_idx = [
        i for i, l in enumerate(seq_lengths) if l > (training_length + 20)
    ]

    new_texts = []
    new_sequences = []

    # Only keep sequences with more than training length tokens
    for i in over_idx:
        new_texts.append(texts[i])
        new_sequences.append(sequences[i])

    training_seq = []
    labels = []

    # Iterate through the sequences of tokens
    for seq in new_sequences:

        # Create multiple training examples from each sequence
        for i in range(training_length, len(seq)):
            # Extract the features and label
            extract = seq[i - training_length:i + 1]

            # Set the features and label
            training_seq.append(extract[:-1])
            labels.append(extract[-1])

    print(f'There are {len(training_seq)} training sequences.')

    # Return everything needed for setting up the model
    return word_idx, idx_word, num_words, word_counts, new_texts, new_sequences, training_seq, labels

In [97]:
data_try["com"]=data_try["question"]+" "+data_try["answer"]

In [98]:
tokenized_corpus = [doc.split(" ") for doc in data_try["com"]]

In [99]:
import re
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(tokenized_corpus)

In [100]:
encoded_docs = t.texts_to_matrix(tokenized_corpus, mode='count')

In [101]:
encoded_docs.shape
import keras 
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(encoded_docs)
print(X.shape)
data_try['sentiment'] = ['pos' if (x==1) else 'neg' for x in data_try['correct']]

(1000, 11305)


In [55]:
import re
from keras.preprocessing.text import Tokenizer
#I have considered a rating above 3 as positive and less than or equal to 3 as negative.
data_try['sentiment'] = ['pos' if (x==1) else 'neg' for x in data_try['correct']]
data_try['com'] = data_try['com'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# for idx,row in data_try.iterrows():
#     row[0] = row[0].replace('rt',' ')
data_try['com'] = [x.encode('ascii') for x in data_try['com']]

tokenizer = Tokenizer(nb_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(data_try['com'][1])
#print(tokenizer.word_index)  # To see the dicstionary
X = tokenizer.texts_to_sequences(data_try['com'].values)
X = pad_sequences(X)

TypeError: cannot use a string pattern on a bytes-like object

In [51]:
data_try['com'][1]

b'Today there is a growing community of more than 2100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal to redefine success in business Join the Movement'

In [81]:
embed_dim = 128
lstm_out = 200
batch_size = 32
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout = 0.2))
# model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(10, 248)))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  del sys.path[0]


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 1757, 128)         320000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 200)               263200    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 402       
Total params: 583,602
Trainable params: 583,602
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Y = pd.get_dummies(data_try['sentiment']).values
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
Y=encoder.fit_transform(data_try['sentiment'])

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y,test_size = 0.20, random_state = 36)


model.fit(X_train, Y_train, batch_size =32, nb_epoch = 1,  verbose = 2)

C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/1


In [ ]:
s,ev=model.evaluate(X_valid,Y_valid,verbose=2,batch_size=32)

In [ ]:
s

In [ ]:
ev

In [44]:
tokenized_corpus = [doc.split(" ") for doc in "hi how are you"]

In [45]:
import re
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(tokenized_corpus)

In [46]:
encoded_docs = t.texts_to_matrix(tokenized_corpus, mode='count')
encoded_docs

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [102]:
embed_dim = 128
lstm_out = 200
batch_size = 32
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import TimeDistributed
from keras.layers import Bidirectional


model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1], dropout = 0.2))
# model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(10, 248)))
model.add(Bidirectional(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\Users\Ashwin Ram\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  app.launch_new_instance()


Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 11305, 128)        320000    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 400)               526400    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 401       
Total params: 846,801
Trainable params: 846,801
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
X.shape

(100, 1757)